In [1]:
import pandas as pd
root = "/homes/syli/dataset/EC_all/all_frame/new"
info_df = pd.read_csv(root+"/total_info.csv", encoding= 'gb2312')
info_df["CaseName"] = info_df["CaseName"].astype(dtype="str")
info_df.head()

,CaseName,LNM,出院日期,发病年龄,身高,体重,病理类型,LVSI,BMI,病理,最大径,分化程度,肌层浸润,FIGO,浸润宫颈,盆腔淋巴,转移腹主动脉淋巴结旁数目,绝经
0,y00184782,1,20200512.0,66.0,NaN,NaN,0,0,NaN,999999,40.0,宫底及双侧宫角,0,NaN,3,0.0,无,1
1,y00216636,1,20200713.0,56.0,165.0,75.0,0,1,27.548209,999999,59.0,宫腔弥漫,2,NaN,3,0.0,双侧盆腔、左侧前哨、左侧髂总,0
2,y00197978,1,20200610.0,67.0,162.0,50.0,0,1,19.051974,999999,63.0,宫腔,0,NaN,999999,0.0,双侧髂总、左侧盆腔、左侧前哨淋巴结、右侧盆腔肿大淋巴结,1
3,y00155653,1,20191224.0,36.0,NaN,NaN,0,1,NaN,999999,55.0,左侧宫角及宫底部深肌层,0,NaN,3,0.0,左侧盆腔、右侧腹股沟,0
4,y00198877,1,20200701.0,60.0,NaN,NaN,0,1,NaN,999999,33.0,宫腔,1,NaN,999999,0.0,右侧盆腔,1


In [2]:
df1 = pd.read_excel(root+"/2013-2018EC data patientsegmented.xlsx")[["CaseName", "术后病理", "病案号", "姓名"]]
df2 = pd.read_excel(root+"/lnm_positive_new.xlsx")[["CaseName", "术后病理", "病案号", "姓名", "门诊号码"]]
df = pd.concat((df1, df2))
df["CaseName"] = df["CaseName"].astype(dtype="str")
print(len(df))
df.head()

847


,CaseName,术后病理,病案号,姓名,门诊号码
0,5436621,一、全子宫：1.子宫内膜复杂不典型增生过长，局部内膜样腺癌Ⅰ级，癌局限于子宫粘膜内，向下未达...,719792,陈海花,NaN
1,5634885,一、全子宫：1.子宫内膜样癌Ⅰ级，大小3.5×2.5cm，浸润子宫浅肌层，向下未累及宫颈。_...,1010226,丛美娟,NaN
2,5618570,"一、全子宫：1.子宫内膜样腺癌Ⅰ级,浸润子宫浅肌层，下缘累及宫颈管粘膜腺体。2.慢性宫颈炎。...",1006869,宋金萍,NaN
3,5599967,一、全子宫：1.子宫内膜样腺癌，I级，浸润子宫浅肌层，向下未累及宫颈内口。周围子宫内膜复杂不...,1002846,朱秀芳,NaN
4,5630892,一、全子宫：1.子宫内膜样癌Ⅱ级，病灶1.8×0.8cm，浸润浅肌层，下缘未达宫颈内口，周围...,1009348,李小微,NaN


In [3]:
new_df = pd.merge(df, info_df)
new_df.drop_duplicates(subset=['CaseName'],keep='first',inplace=True)
len(new_df)

567

In [12]:
set(info_df["CaseName"].values).difference(set(new_df["CaseName"]))
for i,j in zip(info_df.groupby(["CaseName"]).size(), info_df.groupby(["CaseName"]).size().index):
    if i == 2:
        print(j)
info_df.drop_duplicates(subset=['CaseName'],keep='first',inplace=True)
# len(info_df)

5508549


In [5]:
import numpy as np
lvsi = []
for index, i in enumerate(new_df["术后病理"].values):
    if not isinstance(i, str):
        lvsi.append(2)
    elif ("脉管内" in i) and ("癌栓" in i):
        lvsi.append(1)
    else:
        lvsi.append(0)

print(lvsi.count(1), lvsi.count(0), lvsi.count(2))
del new_df["LVSI"]
new_df["LVSI"] = lvsi
# new_df = new_df[new_df["LVSI"] == 2]
new_df.to_csv(root+"/缺失.csv", index=False, encoding="utf_8_sig")
len(new_df)


99 314 154


567

In [18]:
queshi_df = new_df[new_df["LVSI"] == 2]
del queshi_df["LVSI"]
buchong1 = pd.read_excel(root+"/缺失老住院号1.xlsx")[["CaseName", "LVSI"]]
buchong2 = pd.read_excel(root+"/缺失新住院号2.xlsx")[["CaseName", "LVSI"]]

buchong1["CaseName"] = buchong1["CaseName"].astype(dtype="str")
buchong2["CaseName"] = buchong2["CaseName"].astype(dtype="str")
buchong = pd.concat((buchong1, buchong2))
buchong.drop_duplicates(subset=['CaseName'],keep='first',inplace=True)
queshi = pd.merge(queshi_df, buchong, on=["CaseName"], how="left")

In [30]:
info_df = pd.read_csv(root+"/整理新表.csv")
info_df = info_df.dropna(axis=0,subset = ["LVSI"])[["CaseName", "LVSI"]]
info_df.head()

,CaseName,LVSI
0,5618570,0.0
1,5599967,0.0
2,5630892,0.0
3,5615159,1.0
4,5649501,0.0


In [32]:
lnm_df = pd.read_csv("/homes/syli/dataset/EC_all/LNM_model/liunei/T2/test_numeric_feature.csv")
df = pd.merge(info_df, lnm_df)

df.to_csv("./atuo_radiomics/demo_test.csv", index=False)
